In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo

LONDON = ZoneInfo("Europe/London")
now = datetime.now(LONDON)
print(now, now.tzinfo)

2026-02-02 09:15:37.206603+00:00 Europe/London


In [2]:
from datetime import datetime
from zoneinfo import ZoneInfo

from masphd.darwin.extract_segments import extract_segments
from masphd.darwin.realtime_filter import filter_segments_by_now

LONDON = ZoneInfo("Europe/London")

def on_decoded(forecasts, schedules, xml_bytes):
    now = datetime.now(LONDON)

    segs = extract_segments(forecasts, schedules, tz=LONDON, drop_wrong_direction=True)

    # For "in progress" filtering we need planned arrival at SECOND station
    for s in segs:
        loc_second = s.get("loc_second") or {}
        s["planned_arr_second"] = loc_second.get("pta") or loc_second.get("wta")

    segs = filter_segments_by_now(
        segs,
        now=now,
        tz=LONDON,
        mode="in_progress",
        dep_grace_after_now_mins=5,
        arr_grace_before_now_mins=2,
    )

    if not segs:
        return

    # split
    actual_segs = [s for s in segs if s["has_actual_dep"]]
    other_segs = [s for s in segs if not s["has_actual_dep"]]

    print("\n=== LIVE SEGMENTS (in progress) ===", now.strftime("%Y-%m-%d %H:%M:%S"))
    print("Total:", len(segs), "| With actual dep:", len(actual_segs), "| Without actual dep:", len(other_segs))

    if actual_segs:
        rid = actual_segs[0]["rid"]
        print(f"\n--- CONFIRMED ACTUAL DEPARTURE (save to DB later) | rid {rid} ---")
        for s in actual_segs:
            dd = s["departure_delay_min"]
            dw = s["dwell_delay_min"]
            print(
                f'{s["first"]}->{s["second"]} | planned_dep={s["planned_dep"]} '
                f'actual_dep={s["actual_dep_confirmed"]} | dep_delay_min={("NA" if dd is None else f"{dd:.1f}")} '
                f'| dwell_delay_min={("NA" if dw is None else f"{dw:.1f}")}'
            )

    if other_segs:
        rid = other_segs[0]["rid"]
        print(f"\n--- ESTIMATE / MISSING ACTUAL (still predict, show separately) | rid {rid} ---")
        for s in other_segs:
            dd = s["departure_delay_min"]
            dw = s["dwell_delay_min"]
            print(
                f'{s["first"]}->{s["second"]} | planned_dep={s["planned_dep"]} '
                f'best_dep={s["dep_time_for_prediction"]} ({s["dep_time_kind"]}) '
                f'| dep_delay_min={("NA" if dd is None else f"{dd:.1f}")} '
                f'| dwell_delay_min={("NA" if dw is None else f"{dw:.1f}")}'
            )


In [4]:
from masphd.darwin.client import DarwinClient

client = DarwinClient(on_decoded=on_decoded)
client.connect()
client.run_for(300)


=== LIVE SEGMENTS (in progress) === 2026-02-02 09:23:39
Total: 1 | With actual dep: 0 | Without actual dep: 1

--- ESTIMATE / MISSING ACTUAL (still predict, show separately) | rid 202602028061516 ---
BKNHRST->SOTON | planned_dep=09:11 best_dep=09:36 (estimate) | dep_delay_min=25.0 | dwell_delay_min=0.0

=== LIVE SEGMENTS (in progress) === 2026-02-02 09:23:45
Total: 1 | With actual dep: 0 | Without actual dep: 1

--- ESTIMATE / MISSING ACTUAL (still predict, show separately) | rid 202602028061516 ---
BKNHRST->SOTON | planned_dep=09:11 best_dep=09:37 (estimate) | dep_delay_min=26.0 | dwell_delay_min=0.0

=== LIVE SEGMENTS (in progress) === 2026-02-02 09:23:45
Total: 2 | With actual dep: 0 | Without actual dep: 2

--- ESTIMATE / MISSING ACTUAL (still predict, show separately) | rid 202602028061452 ---
BOMO->POKSDWN | planned_dep=09:22 best_dep=09:23 (estimate) | dep_delay_min=1.0 | dwell_delay_min=-4.0
POKSDWN->CHRISTC | planned_dep=09:26 best_dep=09:27 (estimate) | dep_delay_min=1.0 | d

Disconnected. Sleeping 15 seconds.


Sample output of actuals:
```
=== ACTUAL-ONLY DEPARTURES === 2026-02-01 15:40:17
rid: 202602018061460 segments: 1
PSTONE->BRANKSM | planned_dep=15:37 actual_dep=15:39 | dep_delay_min=2.0
Disconnected. Sleeping 15 seconds.


=== ACTUAL-ONLY DEPARTURES === 2026-02-01 15:53:36
rid: 202602018061467 segments: 1
UPWEY->DRCHS | planned_dep=15:52 actual_dep=15:53 | dep_delay_min=1.0

=== ACTUAL-ONLY DEPARTURES === 2026-02-01 15:55:09
rid: 202602017672619 segments: 1
POOLE->PSTONE | planned_dep=15:55 actual_dep=15:54 | dep_delay_min=-1.0


=== ACTUAL-ONLY DEPARTURES === 2026-02-01 16:00:10
rid: 202602017672617 segments: 1
SOTON->SOTPKWY | planned_dep=15:55 actual_dep=15:59 | dep_delay_min=4.0
```